In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Add, Conv2DTranspose, Cropping2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

#1 Dataset 


In [ ]:
#Data
import numpy as np
import os
import cv2
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:
!kaggle datasets download -d andrewmvd/liver-tumor-segmentation

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
'''
X_val = np.load('images_val.npy')
y_val = np.load('masks_val.npy')
X = np.load('images.npy')
y = np.load('masks.npy')
    

print(X_val.shape)
print(y_val.shape)
print(X.shape)
print(y.shape)
'''
if not os.path.isdir("mask"):
  !unzip data.zip

input_path = os.listdir("img/")

val_amount = 0.25
toTrain = round(len(input_path)*(1-val_amount))
toVal = round(len(input_path)*val_amount)

X =  np.zeros((toTrain, 512, 512), dtype = "uint8")
y =  np.zeros((toTrain, 512, 512), dtype = "bool")
X_val = np.zeros((toVal, 512, 512), dtype = "uint8")
y_val = np.zeros((toVal, 512, 512), dtype = "bool")

norm = 0
vali = 0
for i in range(len(input_path)):
    img_path = os.path.join("img/", input_path[i])
    mask_path = os.path.join("mask/", input_path[i])
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2GRAY)
    mask = cv2.cvtColor(cv2.imread(mask_path), cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (512, 512))
    mask = cv2.resize(mask, (512, 512))
    
    if i >= toTrain:
        X_val[vali, :, :] = img
        y_val[vali, :, :] = mask
        vali += 1
    else:
        X[norm, :, :] = img
        y[norm, :, :] = mask
        norm += 1


In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
f, axarr = plt.subplots(2,2)
axarr[0,0].imshow(X[1], cmap = 'gray')
axarr[0,1].imshow(y[1,:,:], cmap = 'gray')
axarr[1,0].imshow(X_val[1], cmap = 'gray')
axarr[1,1].imshow(y_val[1,:,:], cmap = 'gray')

plt.show()

#2 Arquitetura


In [ ]:
filters = 16

In [ ]:
shape = (512,512,3)
input = Input(shape)

In [ ]:
#VGG-16 Encoder
conv1 = Conv2D(filters, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(input)
conv1 = Conv2D(filters, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(conv1)
pool1 = MaxPooling2D((2,2), strides = (2,2), padding = 'same')(conv1)

pool1.shape

TensorShape([None, 256, 256, 16])

In [ ]:
conv2 = Conv2D(filters*2, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(pool1)
conv2 = Conv2D(filters*2, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(conv2)
pool2 = MaxPooling2D((2,2), strides = (2,2), padding = 'same')(conv2)

pool2.shape

TensorShape([None, 128, 128, 32])

In [ ]:
conv3 = Conv2D(filters*4, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(pool2)
conv3 = Conv2D(filters*4, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(conv3)
conv3 = Conv2D(256, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(conv3)
pool3 = MaxPooling2D((2,2), strides = (2,2), padding = 'same')(conv3)

pool3.shape

TensorShape([None, 64, 64, 256])

In [ ]:
conv4 = Conv2D(filters*8, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(pool3)
conv4 = Conv2D(filters*8, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(conv4)
conv4 = Conv2D(filters*8, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(conv4)
pool4 = MaxPooling2D((2,2), strides = (2,2), padding = 'same')(conv4)

pool4.shape

TensorShape([None, 32, 32, 128])

In [ ]:
conv5 = Conv2D(filters*8, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(pool4)
conv5 = Conv2D(filters*8, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(conv5)
conv5 = Conv2D(filters*8, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(conv5)
pool5 = MaxPooling2D((2,2), strides = (2,2), padding = 'same')(conv5)

pool5.shape

TensorShape([None, 16, 16, 128])

In [ ]:
conv6 = Conv2D(filters*64, (7,7), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(pool5)
conv6 = Conv2D(filters*64, (1,1), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(conv6)
conv6 = Conv2D(2, (1,1), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform')(conv6)

conv6.shape

TensorShape([None, 16, 16, 2])

In [ ]:
conv7 = Conv2DTranspose(2, kernel_size=(4,4), strides=(2,2), padding='valid')(conv6)
conv7 = Cropping2D((1,1))(conv7)

conv7.shape

TensorShape([None, 32, 32, 2])

In [ ]:
add01 = Conv2D(2, (1,1), activation = "relu", padding = 'same')(pool4)
add01.shape

TensorShape([None, 32, 32, 2])

In [ ]:
out1 = Add()([conv7, add01])

out1.shape

TensorShape([None, 32, 32, 2])

In [ ]:
out2 = Conv2DTranspose(2, kernel_size=(4,4), strides=(2,2), padding='valid')(out1)
out2 = Cropping2D((1,1))(out2)

out2.shape

TensorShape([None, 64, 64, 2])

In [ ]:
add01 = Conv2D(2, (1,1), activation = "relu", padding = 'same')(pool3)
add01.shape

TensorShape([None, 64, 64, 2])

In [ ]:
out2 = Add()([out2, add01])

out2.shape

TensorShape([None, 64, 64, 2])

In [ ]:
output = Conv2DTranspose(1, kernel_size=(16,16),strides = (8,8), padding='valid',  activation = 'sigmoid')(out2)
output.shape

TensorShape([None, 520, 520, 1])

In [ ]:
output = Cropping2D(cropping=((0,8),(0,8)))(output)
output.shape

TensorShape([None, 512, 512, 1])

In [ ]:
model = Model(inputs = [input], outputs = [output])
#Now using MeanIoU instead of accuracy
#model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3), loss = 'binary_crossentropy', metrics = ['accuracy', MeanIoU(num_classes=2, name = 'MeanIoU')])
#Using only dice
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3), loss = ['binary_crossentropy'], metrics = ['accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_54 (Conv2D)             (None, 512, 512, 16  448         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_55 (Conv2D)             (None, 512, 512, 16  2320        ['conv2d_54[0][0]']              
                                )                                                           

#3 Treinamento


In [ ]:
checkpoint = ModelCheckpoint(filepath='model.h5', save_freq = 'epoch')

#model.load_weights("model.h5")
history = model.fit(X, y, epochs=50, batch_size = 8, callbacks = [checkpoint], validation_data=(X_val, y_val))

Epoch 1/50
53/53 [==============================] - 809s 15s/step - loss: 0.8803 - accuracy: 0.7446 - val_loss: 0.6817 - val_accuracy: 0.7225
Epoch 2/50
53/53 [==============================] - 814s 15s/step - loss: 0.6739 - accuracy: 0.7547 - val_loss: 0.6712 - val_accuracy: 0.7225
Epoch 3/50
53/53 [==============================] - 789s 15s/step - loss: 0.6623 - accuracy: 0.7547 - val_loss: 0.6616 - val_accuracy: 0.7225
Epoch 4/50
53/53 [==============================] - 807s 15s/step - loss: 0.6515 - accuracy: 0.7547 - val_loss: 0.6529 - val_accuracy: 0.7225
Epoch 5/50
53/53 [==============================] - 802s 15s/step - loss: 0.6414 - accuracy: 0.7547 - val_loss: 0.6446 - val_accuracy: 0.7225
Epoch 6/50
53/53 [==============================] - 774s 15s/step - loss: 0.6316 - accuracy: 0.7547 - val_loss: 0.6365 - val_accuracy: 0.7225
Epoch 7/50
53/53 [==============================] - 748s 14s/step - loss: 0.6215 - accuracy: 0.7547 - val_loss: 0.6282 - val_accuracy: 0.7225
Epoch 

In [ ]:
#model = tf.keras.models.load_model('model.31.h5')
a = X_val[0:10]
test = model.predict(a)
#X_val.shape

In [ ]:
image_id = 6

f, axarr = plt.subplots(2,2)
axarr[0,0].imshow(X_val[image_id])
axarr[0,1].imshow(y_val[image_id,:,:], cmap = 'gray')
axarr[1,0].imshow(X_val[image_id])
axarr[1,1].imshow(test[image_id,:,:], cmap = 'gray')

plt.show()

In [ ]:
import pickle


with open('trainHistoryDict', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)